In [7]:
# !pip install seaborn
# !pip install openpyxl

In [8]:
import glob
import pandas as pd
import os
import matplotlib.pyplot as plt


In [9]:
data_clean_folder = "stage3_data_cleaning/v2"
folder_path = data_clean_folder
xlsx_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
xlsx_files

['stage3_data_cleaning/v2\\type10_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type1_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type2_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type3_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type4_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type5_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type6_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type7_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type8_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\type9_label_merged_final_decoded_clean3.xlsx',
 'stage3_data_cleaning/v2\\~$type10_label_merged_final_decoded_clean3.xlsx']

In [10]:
df=pd.read_excel(xlsx_files[1])
df.columns

Index(['label', 'timestamp', 'can_id', 'data_length', 'source_node_id_decimal',
       'service_flag', 'priority', 'message_type_decimal',
       'destination_node_id_decimal', 'request_or_response',
       'service_type_id_decimal', 'start_of_message', 'end_of_message',
       'single_message_frame', 'transfer_ID', 'effective_data_0',
       'effective_data_1', 'effective_data_2', 'effective_data_3',
       'effective_data_4', 'effective_data_5', 'effective_data_6'],
      dtype='object')

In [11]:
df.head()

,label,timestamp,can_id,data_length,source_node_id_decimal,service_flag,priority,message_type_decimal,destination_node_id_decimal,request_or_response,...,end_of_message,single_message_frame,transfer_ID,effective_data_0,effective_data_1,effective_data_2,effective_data_3,effective_data_4,effective_data_5,effective_data_6
0,0,0.000000,10015501,8,1,0,16,341,-99,-99,...,1,1,0,0,0,0,0,8,0,0
1,0,0.192053,104E2001,2,1,0,16,20000,-99,-99,...,1,1,0,0,-199,-199,-199,-199,-199,-199
2,0,0.192335,1F043901,8,1,0,31,1081,-99,-99,...,0,0,0,0,0,0,246,0,-199,-199
3,0,0.192504,1F043901,8,1,0,31,1081,-99,-99,...,0,0,0,0,248,0,0,247,255,223
4,0,0.192637,1F043901,4,1,0,31,1081,-99,-99,...,1,0,0,254,0,0,-199,-199,-199,-199


In [12]:
min(df['timestamp']),max(df['timestamp']), 0.6 * max(df['timestamp']),0.8 *max(df['timestamp'])

(0.0, 179.061121, 107.43667260000001, 143.2488968)

In [20]:
import math
math.floor(0.6 * max(df['timestamp'])),math.ceil(0.6 * max(df['timestamp']))

(107, 108)

In [21]:
df.shape

(207858, 22)

In [37]:
df_train = df[df['timestamp'] <= math.ceil(0.6 * max(df['timestamp']))]
df_val = df[(df['timestamp'] > math.ceil(0.6 * max(df['timestamp']))) & (df['timestamp'] <= math.ceil(0.8 * max(df['timestamp']))) ]
df_test = df[(df['timestamp'] > math.ceil(0.8 * max(df['timestamp']))) & (df['timestamp'] <= max(df['timestamp'])) ]
df_train.shape,df_val.shape,df_test.shape

((116922, 22), (52591, 22), (38345, 22))

In [38]:
df_train.shape[0]+df_val.shape[0]+df_test.shape[0]

207858

In [39]:
df_train['dataset'] = 'train'
df_test['dataset'] = 'test'
df_val['dataset'] = 'val'

C:\Users\Reek\AppData\Local\Temp\ipykernel_14028\1862731007.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['dataset'] = 'train'
C:\Users\Reek\AppData\Local\Temp\ipykernel_14028\1862731007.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['dataset'] = 'test'
C:\Users\Reek\AppData\Local\Temp\ipykernel_14028\1862731007.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [42]:
df = pd.concat([df_train,df_val, df_test], axis=0)

In [43]:
df['dataset'].value_counts()

dataset
train    116922
val       52591
test      38345
Name: count, dtype: int64

In [44]:
df.groupby('dataset').nunique().T

dataset,test,train,val
label,2,2,2
timestamp,38345,116922,52591
can_id,14,38,13
data_length,6,8,4
source_node_id_decimal,5,6,5
service_flag,1,2,1
priority,5,6,4
message_type_decimal,7,9,6
destination_node_id_decimal,1,6,1
request_or_response,1,3,1


In [45]:
df.groupby('dataset')['label'].mean()

dataset
test     0.505672
train    0.545475
val      0.639805
Name: label, dtype: float64

In [52]:
df.groupby(['dataset','can_id']).nunique().T

dataset                         test                                      \
can_id                      01000401 05040601 10015501 1001557A 1001557B   
label                              1        2        1        1        1   
timestamp                         35    35826       35       35       35   
data_length                        1        2        1        1        1   
source_node_id_decimal             1        1        1        1        1   
service_flag                       1        1        1        1        1   
priority                           1        1        1        1        1   
message_type_decimal               1        1        1        1        1   
destination_node_id_decimal        1        1        1        1        1   
request_or_response                1        1        1        1        1   
service_type_id_decimal            1        1        1        1        1   
start_of_message                   1        2        1        1        1   
end_of_message                     1        2        1        1        1   
single_message_frame               1        1        1        1        1   
transfer_ID                       32       32       32       32       32   
effective_data_0                  33      256       35       35       35   
effective_data_1                  33       34        1        1        1   
effective_data_2                  35      119        1        1        1   
effective_data_3                   3       25        1        1        1   
effective_data_4                   1       17        1        1        1   
effective_data_5                   1        2        1        1        1   
effective_data_6                   1        1        1        1        1   

dataset                                                                   ...  \
can_id                      1001557C 1001557D 10040A7A 10040A7B 10040A7C  ...   
label                              1        1        1        1        1  ...   
timestamp                         35       35      498      495      498  ...   
data_length                        1        1        2        2        2  ...   
source_node_id_decimal             1        1        1        1        1  ...   
service_flag                       1        1        1        1        1  ...   
priority                           1        1        1        1        1  ...   
message_type_decimal               1        1        1        1        1  ...   
destination_node_id_decimal        1        1        1        1        1  ...   
request_or_response                1        1        1        1        1  ...   
service_type_id_decimal            1        1        1        1        1  ...   
start_of_message                   1        1        2        2        2  ...   
end_of_message                     1        1        2        2        2  ...   
single_message_frame               1        1        1        1        1  ...   
transfer_ID                       32       32       32       32       32  ...   
effective_data_0                  35       35        7        3        8  ...   
effective_data_1                   1        1       38       32       41  ...   
effective_data_2                   1        1       23       21       29  ...   
effective_data_3                   1        1        4        4        4  ...   
effective_data_4                   1        1        9        9        9  ...   
effective_data_5                   1        1       80       40       50  ...   
effective_data_6                   1        1        7        4        8  ...   

dataset                          val                                      \
can_id                      1001557A 1001557B 1001557C 1001557D 10040A7A   
label                              1        1        1        1        1   
timestamp                         36       36       36       36      402   
data_length                        1        1        1        1        2   
source_

In [47]:
df.head(5)

,label,timestamp,can_id,data_length,source_node_id_decimal,service_flag,priority,message_type_decimal,destination_node_id_decimal,request_or_response,...,single_message_frame,transfer_ID,effective_data_0,effective_data_1,effective_data_2,effective_data_3,effective_data_4,effective_data_5,effective_data_6,dataset
0,0,0.000000,10015501,8,1,0,16,341,-99,-99,...,1,0,0,0,0,0,8,0,0,train
1,0,0.192053,104E2001,2,1,0,16,20000,-99,-99,...,1,0,0,-199,-199,-199,-199,-199,-199,train
2,0,0.192335,1F043901,8,1,0,31,1081,-99,-99,...,0,0,0,0,0,246,0,-199,-199,train
3,0,0.192504,1F043901,8,1,0,31,1081,-99,-99,...,0,0,0,248,0,0,247,255,223,train
4,0,0.192637,1F043901,4,1,0,31,1081,-99,-99,...,0,0,254,0,0,-199,-199,-199,-199,train
